In [1]:
import os 

In [2]:
%pwd


'g:\\MLOps_Project\\MLOps_Project\\research'

In [3]:
os.chdir("../")

In [4]:
import os

# change to your project root
os.chdir(r"G:\MLOps_Project\MLOps_Project")

# verify
print("Current working dir:", os.getcwd())


Current working dir: G:\MLOps_Project\MLOps_Project


In [5]:
%pwd

'G:\\MLOps_Project\\MLOps_Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from mlProject.constants import *
# from mlProject.constants import contant
from mlProject.utils.common import read_yaml, create_directories

In [8]:
from pathlib import Path

import os

ROOT_DIR = Path(os.getcwd())   # current notebook/project root
CONFIG_FILE_PATH = ROOT_DIR / "config" / "config.yaml"
PARAMS_FILE_PATH = ROOT_DIR / "params.yaml"
SCHEMA_FILE_PATH = ROOT_DIR / "schema.yaml"


In [9]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-06 17:04:27,408: INFO: common: yaml file: G:\MLOps_Project\MLOps_Project\config\config.yaml loaded successfully]
[2025-09-06 17:04:27,518: INFO: common: yaml file: G:\MLOps_Project\MLOps_Project\params.yaml loaded successfully]
[2025-09-06 17:04:27,522: INFO: common: yaml file: G:\MLOps_Project\MLOps_Project\schema.yaml loaded successfully]
[2025-09-06 17:04:27,530: INFO: common: created directory at: artifacts]
[2025-09-06 17:04:27,535: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-06 17:04:29,233: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 556
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "84b9e7966f2735df18205bdc4d6ed3d825cfe4e00c8f0055337f04e507a4a731"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block

In [ ]:
import os